In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, json, math, warnings, logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

try:
    import yfinance as yf
    import matplotlib.pyplot as plt
except Exception:
    pass

warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("fib_macd_pullback")

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    # Data
    start_date: str = "2015-01-01"
    end_date: str   = "2025-01-01"
    static_symbols: Optional[List[str]] = None
    static_symbols_path: Optional[str] = None
    cache_dir: str = "cache"
    out_dir: str   = "outputs"
    plot: bool     = True

    # --- Strategy (from the image) ---
    ema_fast: int = 50                 # uptrend filter
    ema_slow: int = 100                # uptrend filter
    swing_high_lookback: int = 20      # "make a swing-high" = 20D high
    swing_low_lookback: int = 40       # look back to find leg's low
    fib_tolerance_pct: float = 0.003   # tolerance around fib zone (0.3%)
    macd_fast: int = 12
    macd_slow: int = 26
    macd_signal: int = 9
    macd_cross_lookback: int = 5       # "recent" cross-up allowed within last K days
    rr_target: float = 2.0             # at least 1:2 R:R

    # Portfolio / risk
    stop_at_pullback_low: bool = True  # per text
    apply_fees: bool    = True
    initial_capital: float = 500_000.0
    max_concurrent_positions: int = 5
    deploy_cash_frac: float = 0.25     # cap daily deployment to 25% of available cash

    # Execution
    entry_on_next_open: bool = True
    exit_on_next_open: bool = True

    # Candidate ranking & filters (kept; works with portfolio engine)
    benchmark_try: Tuple[str,...] = ("^CNX500","^CRSLDX","^NSE500","^NIFTY500","^BSE500","^NSEI")
    volar_lookback: int = 252
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.50
    top_k_daily: int = 300

    # Optional guards
    enable_basic_liquidity: bool = False
    min_price_inr: float = 50.0
    min_avg_vol_20d: float = 50000.0

CFG = Config()

# =========================
# FEES (per user spec)
# =========================
APPLY_FEES = True
def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not APPLY_FEES:
        return 0.0
    BROKER_PCT = 0.001
    BROKER_MIN = 5.0
    BROKER_CAP = 20.0
    STT_PCT = 0.001
    STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297
    SEBI_PCT = 0.000001
    IPFT_PCT = 0.000001
    GST_PCT = 0.18
    DP_SELL = 20.0 if turnover_sell >= 100 else 0.0

    def _broker(turnover):
        if turnover <= 0:
            return 0.0
        fee = turnover * BROKER_PCT
        fee = max(BROKER_MIN, min(fee, BROKER_CAP))
        return fee

    br_buy  = _broker(turnover_buy)
    br_sell = _broker(turnover_sell)
    stt   = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    exch  = EXCH_PCT * (turnover_buy + turnover_sell)
    sebi  = SEBI_PCT * (turnover_buy + turnover_sell)
    ipft  = IPFT_PCT * (turnover_buy + turnover_sell)
    dp    = DP_SELL
    gst_base = br_buy + br_sell + dp + exch + sebi + ipft
    gst   = GST_PCT * gst_base
    return float((br_buy + br_sell) + stt + stamp + exch + sebi + ipft + dp + gst)

# =========================
# Helpers
# =========================
def ensure_dirs(*paths):
    for p in paths:
        os.makedirs(p, exist_ok=True)

def today_str():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y-%m-%d")

def load_static_symbols(static_symbols: Optional[List[str]], static_symbols_path: Optional[str]) -> List[str]:
    syms: List[str] = []
    if static_symbols and len(static_symbols) > 0:
        syms = list(static_symbols)
    elif static_symbols_path and os.path.exists(static_symbols_path):
        with open(static_symbols_path, "r") as f:
            syms = [line.strip() for line in f if line.strip()]
    else:
        raise ValueError(
            "Provide CFG.static_symbols=[...] ('.NS' suffixes) or set CFG.static_symbols_path "
            "to a file containing one symbol per line."
        )
    out = []
    for s in syms:
        s = s.strip().upper()
        if not s.endswith(".NS"):
            s = f"{s}.NS"
        out.append(s)
    seen = set()
    uniq = []
    for s in out:
        if s not in seen:
            uniq.append(s); seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], start: str, end: Optional[str], cache_dir: str) -> Dict[str, pd.DataFrame]:
    ensure_dirs(cache_dir)
    data = {}
    end = end or today_str()
    for ticker in tickers:
        cache_path = os.path.join(cache_dir, f"{ticker.replace('^', '_')}.parquet")
        if os.path.exists(cache_path):
            try:
                df = pd.read_parquet(cache_path)
                if len(df) and pd.to_datetime(df.index[-1]).strftime("%Y-%m-%d") >= end:
                    data[ticker] = df
                    continue
            except Exception:
                pass
        try:
            df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
            if df is None or df.empty:
                continue
            df = df.rename(columns=str.title)
            df = df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
            df.index.name = "date"
            df.to_parquet(cache_path)
            data[ticker] = df
        except Exception:
            continue
    return data

def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

def macd(close: pd.Series, fast=12, slow=26, signal=9) -> pd.DataFrame:
    ema_fast = ema(close, fast)
    ema_slow = ema(close, slow)
    line = ema_fast - ema_slow
    signal_line = ema(line, signal)
    hist = line - signal_line
    return pd.DataFrame({"macd": line, "signal": signal_line, "hist": hist})

# =========================
# INDICATORS for this strategy
# =========================
def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    out = df.copy()
    out["ema_fast"] = ema(out["Close"], cfg.ema_fast)
    out["ema_slow"] = ema(out["Close"], cfg.ema_slow)
    out["uptrend"]  = out["ema_fast"] > out["ema_slow"]
    m = macd(out["Close"], cfg.macd_fast, cfg.macd_slow, cfg.macd_signal)
    out = out.join(m)
    out["macd_cross_up"] = (out["macd"].shift(1) <= out["signal"].shift(1)) & (out["macd"] > out["signal"])
    out["avg_vol_20"] = out["Volume"].rolling(20).mean()
    out["high_52w"]   = out["Close"].rolling(cfg.filter_52w_window).max()
    return out.dropna()

def basic_liquidity_ok(row: pd.Series, cfg: Config) -> bool:
    if not cfg.enable_basic_liquidity:
        return True
    return not (row["Close"] < cfg.min_price_inr or row["avg_vol_20"] < cfg.min_avg_vol_20d)

# =========================
# STRATEGY: Fibonacci Pullback + MACD confirmation (from the image)
# =========================
def simulate_ticker(ticker: str, df: pd.DataFrame, cfg: Config):
    """
    Rules encoded from the image:
      • Trade only in uptrends (EMA50 > EMA100).
      • When a new swing-high forms (20D high), define the prior leg low as the lowest low in the last 40 days.
      • Compute Fib retracements of that leg; watch the 38.2–61.8% zone.
      • If price pulls back into the zone AND a green candle closes within it AND MACD has crossed up recently,
        then BUY above that candle's close (executed next bar open).
      • Stoploss = lowest low since swing-high (pullback low).  Target = max(prior swing-high, entry + 2R).
      • Exit on SL/TP only.
    """
    d = compute_indicators(df, cfg).copy()
    cols = ["ticker","side","date","price","shares","reason","signal_reason","score",
            "macd","signal","ema_fast","ema_slow","close","high_52w"]
    if d.empty:
        return pd.DataFrame(columns=cols), pd.Series(dtype=float)

    idx = list(d.index)
    in_pos = False
    entry_px = stop_px = tgt_px = 0.0

    # state for current swing leg
    leg_low = np.nan
    leg_high = np.nan
    leg_high_dt = None
    fib_lo = fib_hi = np.nan    # 61.8 and 38.2 retracement prices (zone)

    trades = []

    for i in range(cfg.swing_high_lookback, len(idx)-1):
        dt = idx[i]; nxt = idx[i+1]
        row = d.loc[dt]; nxt_row = d.loc[nxt]

        # --- maintain/upsert active swing leg when a new 20D high prints in uptrend
        window = d.iloc[max(0, i - cfg.swing_low_lookback):i+1]
        if row["uptrend"] and row["High"] >= window["High"].rolling(cfg.swing_high_lookback).max().iloc[-1]:
            # new swing-high; set the leg from recent lookback low to this high
            leg_low  = float(window["Low"].min())
            leg_high = float(row["High"])
            leg_high_dt = dt
            # fib zone from leg_high downwards
            diff = leg_high - leg_low
            fib_382 = leg_high - 0.382 * diff
            fib_618 = leg_high - 0.618 * diff
            fib_hi, fib_lo = max(fib_382, fib_618), min(fib_382, fib_618)

        # if we don't have a valid leg, continue
        if not (np.isfinite(leg_low) and np.isfinite(leg_high)):
            continue

        # if price breaks above leg_high, refresh leg to new high
        if row["High"] > leg_high:
            leg_high = float(row["High"])
            leg_high_dt = dt
            diff = leg_high - leg_low
            fib_382 = leg_high - 0.382 * diff
            fib_618 = leg_high - 0.618 * diff
            fib_hi, fib_lo = max(fib_382, fib_618), min(fib_382, fib_618)

        # --- Entry logic when not in position
        if not in_pos:
            # pullback is "active" only after swing-high date
            if leg_high_dt is None or dt <= leg_high_dt:
                continue

            # did candle close within (38.2–61.8)% zone (with tolerance) and is it green?
            tol = cfg.fib_tolerance_pct
            lo_zone = fib_lo * (1 - tol)
            hi_zone = fib_hi * (1 + tol)
            in_zone = (row["Close"] >= lo_zone) and (row["Close"] <= hi_zone)
            green = row["Close"] > row["Open"]

            # recent MACD cross-up?
            recent_macd_cross = d["macd_cross_up"].iloc[max(0, i - cfg.macd_cross_lookback + 1):i+1].any()

            if row["uptrend"] and in_zone and green and recent_macd_cross and basic_liquidity_ok(row, cfg):
                # Enter above the closing price of the green candle -> will fill at next open (gap guard)
                px_signal = float(row["Close"])
                px = float(nxt_row["Open"] if cfg.entry_on_next_open else px_signal)
                trades.append({
                    "ticker": ticker, "side": "BUY", "date": (nxt if cfg.entry_on_next_open else dt),
                    "price": px, "shares": 0, "reason": "fib_macd_entry",
                    "signal_reason": f"Green@Fib(38.2–61.8) + MACD↑ + Uptrend",
                    "score": float(row["Close"] - fib_lo),
                    "macd": float(row["macd"]), "signal": float(row["signal"]),
                    "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]),
                    "close": float(row["Close"]), "high_52w": float(row["high_52w"])
                })
                in_pos = True
                entry_px = px

                # stop = lowest low since swing-high (pullback low)
                post_high_slice = d.loc[leg_high_dt:dt]
                pullback_low = float(post_high_slice["Low"].min()) if len(post_high_slice) else float(row["Low"])
                stop_px = pullback_low if cfg.stop_at_pullback_low else float(row["Low"])
                # ensure stop below entry
                stop_px = min(stop_px, entry_px * 0.999)

                R = entry_px - stop_px
                tgt_rr = entry_px + cfg.rr_target * R
                tgt_px = max(leg_high, tgt_rr)

        else:
            # --- Exit on SL/TP using next bar H/L; priority: if both hit, take target
            exec_date = nxt if cfg.exit_on_next_open else dt
            hit = None
            exec_price = None

            if (nxt_row["Low"] <= stop_px) and (nxt_row["High"] >= tgt_px):
                hit, exec_price = "target", float(tgt_px)
            elif nxt_row["Low"] <= stop_px:
                hit, exec_price = "stop", float(stop_px)
            elif nxt_row["High"] >= tgt_px:
                hit, exec_price = "target", float(tgt_px)

            if hit is not None:
                trades.append({
                    "ticker": ticker, "side": "SELL", "date": exec_date,
                    "price": float(exec_price), "shares": 0, "reason": hit,
                    "signal_reason": "", "score": np.nan,
                    "macd": float(row["macd"]), "signal": float(row["signal"]),
                    "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]),
                    "close": float(row["Close"]), "high_52w": float(row["high_52w"])
                })
                in_pos = False
                entry_px = stop_px = tgt_px = 0.0

    # safety close
    if in_pos:
        last_dt = d.index[-1]; row = d.loc[last_dt]
        trades.append({
            "ticker": ticker, "side": "SELL", "date": last_dt,
            "price": float(row["Close"]), "shares": 0, "reason": "final_close",
            "signal_reason": "", "score": np.nan,
            "macd": float(row["macd"]), "signal": float(row["signal"]),
            "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]),
            "close": float(row["Close"]), "high_52w": float(row["high_52w"])
        })

    cols = ["ticker","side","date","price","shares","reason","signal_reason","score",
            "macd","signal","ema_fast","ema_slow","close","high_52w"]
    return pd.DataFrame(trades, columns=cols), pd.Series(dtype=float)

# =========================
# Benchmark + ranking + sizing (unchanged)
# =========================
def pick_benchmark(benchmarks: Tuple[str,...], start: str, end: Optional[str], cache_dir: str) -> Tuple[str, pd.DataFrame]:
    for t in benchmarks:
        data = fetch_prices([t], start, end, cache_dir)
        df = data.get(t)
        if df is not None and not df.empty:
            log.info("Using benchmark: %s", t)
            return t, df
    idx = pd.date_range(start=start, end=end or today_str(), freq="B")
    df = pd.DataFrame({"Close": np.ones(len(idx))}, index=idx)
    log.warning("No benchmark found; using synthetic flat series.")
    return "SYNTH_BENCH", df

def compute_volar_scores(end_dt: pd.Timestamp, tickers: List[str], data_map: Dict[str,pd.DataFrame], bench_df: pd.DataFrame, lookback: int) -> Dict[str, float]:
    scores = {}
    bser = bench_df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
    for t in tickers:
        df = data_map.get(t)
        if df is None or df.empty:
            scores[t] = 0.0; continue
        if end_dt not in df.index:
            df = df[df.index <= end_dt]
            if df.empty:
                scores[t] = 0.0; continue
        r = df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
        common = pd.concat([r, bser], axis=1, keys=["s","b"]).dropna()
        if common.shape[0] < max(20, int(0.4*lookback)):
            scores[t] = 0.0; continue
        excess = common["s"] - common["b"]
        vol = common["s"].std(ddof=0)
        scores[t] = 0.0 if vol <= 1e-8 else float((excess.mean() / vol) * math.sqrt(252.0))
    return scores

def markowitz_long_only(mu: np.ndarray, Sigma: np.ndarray) -> np.ndarray:
    n = len(mu)
    eps = 1e-6
    Sigma = Sigma + eps*np.eye(n)
    def solve_lambda(lmbd: float, active_mask=None):
        if active_mask is None:
            A = np.block([[2*lmbd*Sigma, np.ones((n,1))],[np.ones((1,n)), np.zeros((1,1))]])
            b = np.concatenate([mu, np.array([1.0])])
            try:
                sol = np.linalg.solve(A, b); w = sol[:n]
            except np.linalg.LinAlgError:
                w = np.full(n, 1.0/n)
            return w
        else:
            idx = np.where(active_mask)[0]
            if len(idx)==0: return np.full(n, 1.0/n)
            S = Sigma[np.ix_(idx, idx)]; o = np.ones(len(idx)); m = mu[idx]
            A = np.block([[2*lmbd*S, o[:,None]],[o[None,:], np.zeros((1,1))]])
            b = np.concatenate([m, np.array([1.0])])
            try:
                sol = np.linalg.solve(A, b); w_sub = sol[:len(idx)]
            except np.linalg.LinAlgError:
                w_sub = np.full(len(idx), 1.0/len(idx))
            w = np.zeros(n); w[idx] = w_sub; return w
    best_w = np.full(n, 1.0/n); best_sr = -1e9
    for lmbd in np.logspace(-3, 3, 31):
        active = np.ones(n, dtype=bool); w=None
        for _ in range(n):
            w = solve_lambda(lmbd, active_mask=active)
            neg = w < 0
            if not neg.any(): break
            worst = np.argmin(w); active[worst] = False
        if w is None: continue
        w = np.clip(w, 0, None); 
        if w.sum() <= 0: continue
        w = w / w.sum()
        mu_p = float(mu @ w); vol_p = float(np.sqrt(w @ Sigma @ w))
        if vol_p <= 1e-8: continue
        sr = mu_p / vol_p
        if sr > best_sr: best_sr = sr; best_w = w.copy()
    return best_w

# =========================
# Portfolio simulation (unchanged)
# =========================
def aggregate_and_apply(all_trades: pd.DataFrame, data_map: Dict[str, pd.DataFrame], bench_df: pd.DataFrame, cfg: Config):
    if all_trades.empty:
        return all_trades, pd.Series(dtype=float), {}

    side_order = {"BUY": 0, "SELL": 1}
    all_trades = (all_trades
        .assign(_sorder=all_trades["side"].map(side_order))
        .sort_values(by=["date", "_sorder"], kind="stable")
        .drop(columns=["_sorder"])
        .reset_index(drop=True)
    )
    all_trades["date"] = pd.to_datetime(all_trades["date"])

    equity_curve = []
    dates = sorted(all_trades["date"].unique().tolist())
    cash = cfg.initial_capital
    open_positions = {}
    completed_legs = []

    global APPLY_FEES
    APPLY_FEES = cfg.apply_fees

    def _get_close_on(tkr, dt):
        df = data_map.get(tkr)
        if df is None or df.empty:
            return np.nan
        if dt in df.index:
            return float(df.loc[dt, "Close"])
        prev = df[df.index <= dt]
        if prev.empty:
            return np.nan
        return float(prev["Close"].iloc[-1])

    if dates:
        seed_date = pd.to_datetime(dates[0]) - pd.Timedelta(days=1)
        equity_curve.append((seed_date, float(cash)))

    for dt in dates:
        day_trades = all_trades[all_trades["date"] == dt].copy()

        # ---- SELL first ----
        for _, tr in day_trades[day_trades["side"] == "SELL"].iterrows():
            tkr = tr["ticker"]; price = float(tr["price"])
            pos = open_positions.get(tkr)
            if pos is None: continue
            shares = int(pos["shares"])
            turnover_sell = shares * price
            fee = calc_fees(0.0, turnover_sell)
            pnl = (price - pos["entry_px"]) * shares
            cash += (turnover_sell - fee)
            realized = pnl - fee - pos.get("buy_fee", 0.0)
            completed_legs.append({
                "ticker": tkr, "side": "SELL", "date": dt, "price": price,
                "shares": shares, "reason": tr.get("reason",""),
                "turnover": turnover_sell, "fees_inr": fee, "pnl_inr": realized,
                "macd": tr.get("macd", np.nan), "signal": tr.get("signal", np.nan),
                "ema_fast": tr.get("ema_fast", np.nan), "ema_slow": tr.get("ema_slow", np.nan),
                "close": tr.get("close", np.nan), "high_52w": tr.get("high_52w", np.nan),
                "volar": tr.get("volar", np.nan), "mvo_weight": np.nan, "alloc_inr": np.nan
            })
            log.info("Exit %-12s px=%8.2f shares=%6d reason=%s net=%.2f cash=%.2f",
                     tkr, price, shares, tr.get("reason",""), realized, cash)
            del open_positions[tkr]

        # ---- BUY candidates today ----
        buys_today = day_trades[day_trades["side"] == "BUY"].copy()
        # 52w filter
        if not buys_today.empty:
            keep = []
            for _, rr in buys_today.iterrows():
                df = data_map.get(rr["ticker"])
                if df is None or df.empty or dt not in df.index:
                    continue
                close = float(df.loc[dt, "Close"])
                hist = df["Close"].loc[:dt]
                window = hist.iloc[-cfg.filter_52w_window:] if len(hist)>=cfg.filter_52w_window else hist
                high_52w = float(window.max())
                if high_52w>0 and close >= cfg.within_pct_of_52w_high * high_52w:
                    keep.append(rr)
            buys_today = pd.DataFrame(keep) if keep else pd.DataFrame(columns=buys_today.columns)

        # Exclude already-held tickers
        if not buys_today.empty:
            buys_today = buys_today[~buys_today["ticker"].isin(open_positions.keys())]

        # VOLAR ranking
        if not buys_today.empty:
            tickers = buys_today["ticker"].tolist()
            volar_scores = compute_volar_scores(dt, tickers, data_map, bench_df, cfg.volar_lookback)
            buys_today["volar"] = buys_today["ticker"].map(volar_scores)
            buys_today = buys_today.sort_values("volar", ascending=False).reset_index(drop=True)

        slots = cfg.max_concurrent_positions - len(open_positions)
        selected = pd.DataFrame(columns=buys_today.columns)
        if slots > 0 and not buys_today.empty:
            selected = buys_today.head(min(cfg.top_k_daily, slots)).copy()

        if not selected.empty:
            log.info("Selected %d BUY candidates on %s:", selected.shape[0], dt.date())
            for i, rr in selected.reset_index(drop=True).iterrows():
                log.info("  %-12s volar=%6.2f rank=%d px=%8.2f", rr["ticker"], rr.get("volar",0.0), i+1, rr["price"])

            # MVO sizing
            names = selected["ticker"].tolist()
            rets = []
            for t in names:
                df = data_map.get(t)
                ser = df["Close"].loc[:dt].pct_change().dropna().iloc[-cfg.volar_lookback:]
                rets.append(ser)
            R = pd.concat(rets, axis=1)
            R.columns = names
            R = R.dropna()
            if R.empty or R.shape[0] < max(20, int(0.4*cfg.volar_lookback)) or R.shape[1] == 0:
                weights = np.full(len(names), 1.0/len(names))
            else:
                mu = R.mean().values
                Sigma = R.cov().values
                weights = markowitz_long_only(mu, Sigma)

            deploy_cash = max(0.0, float(cash)) * float(cfg.deploy_cash_frac)

            if deploy_cash <= 0:
                log.info("No deployable cash (cap=%.0f%%) on %s", 100*cfg.deploy_cash_frac, dt.date())
            else:
                alloc = (weights / weights.sum()) * deploy_cash if weights.sum()>0 else np.full(len(names), deploy_cash/len(names))
                rank_map = {row["ticker"]: (idx+1) for idx, (_, row) in enumerate(selected.iterrows())}
                for w_amt, t in zip(alloc, names):
                    df_t = data_map[t]
                    price = float(df_t.loc[dt, "Close"] if dt in df_t.index else df_t["Close"].loc[:dt].iloc[-1])
                    shares = int(math.floor(w_amt / price))
                    if shares <= 0:
                        log.info("Skip BUY %-12s (alloc %.2f too small)", t, w_amt)
                        continue
                    turn = shares * price
                    fee = calc_fees(turn, 0.0)
                    total_cost = turn + fee
                    if total_cost > cash:
                        shares = int(math.floor((cash - fee) / price))
                        if shares <= 0:
                            log.info("Skip BUY %-12s due to cash/fees", t)
                            continue
                        turn = shares * price
                        total_cost = turn + fee
                    cash -= total_cost
                    open_positions[t] = {"entry_date": dt, "entry_px": price, "shares": shares, "buy_fee": fee, "entry_reason": "entry"}
                    row_sel = selected[selected["ticker"]==t].iloc[0]
                    volar_val = float(row_sel.get("volar", np.nan))
                    rank_pos = rank_map.get(t, np.nan)
                    high_52w = float(row_sel.get("high_52w", np.nan))
                    close_val = float(row_sel.get("close", np.nan))
                    pct_52w = (close_val / high_52w) if (high_52w and high_52w>0) else np.nan
                    mvo_weight_today = (w_amt / deploy_cash) if deploy_cash > 0 else 0.0
                    sig_reason = row_sel.get("signal_reason", "Fib zone + MACD↑")
                    reason_text = (
                        f"{sig_reason}; 52w%={pct_52w:.1%} (>= {CFG.within_pct_of_52w_high:.0%}); "
                        f"VOLAR rank {int(rank_pos)}/{len(names)} (VOLAR={volar_val:.2f}); "
                        f"MVO weight={mvo_weight_today:.1%} of capped cash ({100*cfg.deploy_cash_frac:.0f}% of available)"
                    )
                    completed_legs.append({
                        "ticker": t, "side": "BUY", "date": dt, "price": price,
                        "shares": shares, "reason": reason_text,
                        "turnover": turn, "fees_inr": fee, "pnl_inr": 0.0,
                        "macd": float(row_sel.get("macd", np.nan)), "signal": float(row_sel.get("signal", np.nan)),
                        "ema_fast": float(row_sel.get("ema_fast", np.nan)), "ema_slow": float(row_sel.get("ema_slow", np.nan)),
                        "close": close_val, "high_52w": high_52w,
                        "volar": volar_val, "mvo_weight": float(mvo_weight_today), "alloc_inr": float(w_amt)
                    })
                    log.info("BUY %-12s px=%8.2f sh=%6d fee=%.2f cash=%.2f :: %s",
                             t, price, shares, fee, cash, reason_text)

        # MTM valuation
        mtm = 0.0
        for _tkr, pos in open_positions.items():
            px = _get_close_on(_tkr, dt)
            if not np.isnan(px):
                mtm += pos["shares"] * px
        total_equity = cash + mtm
        equity_curve.append((dt, float(total_equity)))

    eq_ser = pd.Series([e for _, e in equity_curve], index=[d for d, _ in equity_curve])
    legs_df = pd.DataFrame(completed_legs).sort_values(["date", "ticker", "side"]).reset_index(drop=True)

    # Build roundtrips
    roundtrips = []
    by_tkr_open = {}
    for _, leg in legs_df.iterrows():
        tkr = leg["ticker"]
        if leg["side"] == "BUY":
            by_tkr_open[tkr] = leg
        else:
            buy = by_tkr_open.pop(tkr, None)
            if buy is None: continue
            fees_total = float(buy.get("fees_inr", 0.0) + leg.get("fees_inr", 0.0))
            gross_pnl = (leg["price"] - buy["price"]) * buy["shares"]
            net_pnl   = gross_pnl - fees_total
            ret_pct   = (leg["price"] / buy["price"] - 1.0) * 100.0
            days_held = (pd.to_datetime(leg["date"]) - pd.to_datetime(buy["date"])).days
            roundtrips.append({
                "ticker": tkr,
                "entry_date": pd.to_datetime(buy["date"]),
                "entry_price": float(buy["price"]),
                "exit_date": pd.to_datetime(leg["date"]),
                "exit_price": float(leg["price"]),
                "days_held": int(days_held),
                "shares": int(buy["shares"]),
                "entry_reason": buy.get("reason",""),
                "exit_reason": leg.get("reason",""),
                "gross_pnl_inr": float(gross_pnl),
                "fees_total_inr": float(fees_total),
                "net_pnl_inr": float(net_pnl),
                "return_pct": float(ret_pct),
                "macd_entry": float(buy.get("macd", np.nan)),
                "ema_fast_entry": float(buy.get("ema_fast", np.nan)),
                "ema_slow_entry": float(buy.get("ema_slow", np.nan)),
                "high_52w_entry": float(buy.get("high_52w", np.nan)),
                "volar_entry": float(buy.get("volar", np.nan)),
                "mvo_weight_entry": float(buy.get("mvo_weight", np.nan)),
                "alloc_inr_entry": float(buy.get("alloc_inr", np.nan))
            })
    trips_df = pd.DataFrame(roundtrips).sort_values(["entry_date","ticker"]).reset_index(drop=True)

    metrics = compute_metrics(eq_ser, legs_df)
    return legs_df, trips_df, eq_ser, metrics

def compute_metrics(equity: pd.Series, legs_df: pd.DataFrame):
    out = {}
    if equity is None or equity.empty:
        return out
    eq = equity.dropna()
    daily_ret = eq.pct_change().fillna(0.0)

    days = (eq.index[-1] - eq.index[0]).days or 1
    years = days / 365.25
    cagr = (eq.iloc[-1] / eq.iloc[0]) ** (1/years) - 1 if years > 0 else 0.0

    sharpe = (daily_ret.mean() / daily_ret.std(ddof=0) * np.sqrt(252)) if daily_ret.std(ddof=0) > 0 else 0.0
    cummax = eq.cummax()
    dd = (eq - cummax) / cummax
    max_dd = dd.min()

    wins = 0
    n_sells = legs_df[legs_df["side"] == "SELL"].shape[0] if legs_df is not None and not legs_df.empty else 0
    for _, r in legs_df[legs_df["side"] == "SELL"].iterrows():
        if float(r.get("pnl_inr", 0.0)) > 0:
            wins += 1
    win_rate = (wins / n_sells) * 100.0 if n_sells > 0 else 0.0

    out.update({
        "start_equity_inr": float(eq.iloc[0]),
        "final_equity_inr": float(eq.iloc[-1]),
        "cagr_pct": float(cagr * 100),
        "sharpe": float(sharpe),
        "max_drawdown_pct": float(max_dd * 100),
        "win_rate_pct": float(win_rate),
        "n_trades": int(n_sells),
    })
    return out

def plot_equity(equity: pd.Series, out_path: str):
    if equity is None or equity.empty:
        return
    try:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10,5))
        plt.plot(equity.index, equity.values)
        plt.title("Equity Curve")
        plt.xlabel("Date")
        plt.ylabel("Equity (INR)")
        plt.tight_layout()
        plt.savefig(out_path)
        plt.close()
    except Exception:
        pass

def backtest(cfg: Config):
    ensure_dirs(cfg.cache_dir, cfg.out_dir)
    log.info("Universe: loading static symbols...")
    symbols = load_static_symbols(cfg.static_symbols, cfg.static_symbols_path)
    log.info("Loaded %d symbols.", len(symbols))

    log.info("Data: fetching OHLCV from yfinance (adjusted)...")
    data_map = fetch_prices(symbols, cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Downloaded %d symbols with data.", len(data_map))

    bench_tkr, bench_df = pick_benchmark(cfg.benchmark_try, cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Benchmark selected: %s", bench_tkr)

    log.info("Signals: generating Fibonacci + MACD pullback entries...")
    all_trades = []
    for i, tkr in enumerate(symbols, 1):
        df = data_map.get(tkr)
        if df is None or df.empty:
            continue
        tr, _ = simulate_ticker(tkr, df, cfg)
        if not tr.empty:
            all_trades.append(tr)
        if i % 50 == 0:
            log.info("  processed %d/%d tickers...", i, len(symbols))

    if not all_trades:
        log.warning("No signals generated; try loosening swing lookbacks or MACD window.")
        return None, None, None, {}
    all_trades = pd.concat(all_trades, ignore_index=True)

    log.info("Portfolio: cap daily deploy to %.0f%% of cash; 52w>=%.0f%% high; top-%d by VOLAᵣ; MVO; max %d positions.",
             cfg.deploy_cash_frac*100, cfg.within_pct_of_52w_high*100, cfg.top_k_daily, cfg.max_concurrent_positions)
    legs_df, trips_df, equity, metrics = aggregate_and_apply(all_trades, data_map, bench_df, cfg)

    stamp = pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y%m%d_%H%M%S")
    legs_path = os.path.join(cfg.out_dir, f"trades_legs_{stamp}.csv")
    trips_path = os.path.join(cfg.out_dir, f"trades_roundtrips_{stamp}.csv")
    equity_path = os.path.join(cfg.out_dir, f"equity_{stamp}.csv")
    metrics_path = os.path.join(cfg.out_dir, f"metrics_{stamp}.json")
    eq_plot_path = os.path.join(cfg.out_dir, f"equity_{stamp}.png")

    if legs_df is not None:
        legs_df.to_csv(legs_path, index=False)
    if trips_df is not None:
        trips_df.to_csv(trips_path, index=False)
    if equity is not None:
        pd.DataFrame({"date": equity.index, "equity": equity.values}).to_csv(equity_path, index=False)
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=2)

    if cfg.plot and equity is not None:
        plot_equity(equity, eq_plot_path)

    log.info("=== METRICS ===\n%s", json.dumps(metrics, indent=2))
    log.info("Files written:\n  %s\n  %s\n  %s\n  %s", legs_path, trips_path, equity_path, metrics_path)
    if cfg.plot:
        log.info("  %s", eq_plot_path)

def main():
    global APPLY_FEES
    APPLY_FEES = bool(CFG.apply_fees)
    # Set your universe here or via file path.
    # CFG.static_symbols = ['RELIANCE.NS','HDFCBANK.NS','INFY.NS','TCS.NS']
    CFG.static_symbols_path = "nifty500.txt"
    backtest(CFG)

if __name__ == "__main__":
    main()


2025-11-04 13:35:11 | INFO | Universe: loading static symbols...
2025-11-04 13:35:11 | INFO | Loaded 500 symbols.
2025-11-04 13:35:11 | INFO | Data: fetching OHLCV from yfinance (adjusted)...
2025-11-04 13:35:15 | ERROR | 
1 Failed download:
2025-11-04 13:35:15 | ERROR | ['ABLBL.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600, endDate = 1735669800")')
2025-11-04 13:35:19 | ERROR | 
1 Failed download:
2025-11-04 13:35:19 | ERROR | ['AEGISVOPAK.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600, endDate = 1735669800")')
2025-11-04 13:35:21 | ERROR | 
1 Failed download:
2025-11-04 13:35:21 | ERROR | ['AGARWALEYE.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600